In [1]:
#this uses iPython magic to make plots appear inline
import os
import sys
import numpy as np
import subprocess

In [2]:
def count_lines(f):
    lineCount = 0
    with open(f, 'r') as f:
        for line in f:
            lineCount += 1
        return lineCount
def runCMD(cmd):
    val = subprocess.Popen(cmd, shell=True).wait()
    if val == 0:
        pass
    else:
        print ('command failed')
        print (cmd)
        sys.exit(1)

In [3]:
ervDir = '/home/ampend/links/kidd-lab/ampend-projects/cfERVs/'
bedfiles = [ervDir + 'input/cfERV_NonReferenceInsertionSites_2017-06-29.bed',ervDir + 'input/cfERV_ReferenceInsertionSites_2017-06-29.bed']

In [4]:

for File in bedfiles:
    print('#Reading in coordinates from the following ERV insertion site bed file:\n', File)
    f = File
    lineCount = count_lines(f)
    print('#There are a total of %i cfERV insertions in this bed file\n' % (lineCount))
    bedFile = open(f, 'r')

    ervDict = {}

    for line in bedFile:
        line = line.rstrip()
        line = line.split('\t')
        chrom = line[0]
        start = int(line[1])
        end = int(line[2])
        ID = line[3]
        if 'NonReference' in File:
            insertionType = 'NonReference'
        else:
            insertionType = 'Reference'
        ervDict[ID] = [chrom,start,end,ID,'Genes','Olfactory','FST','VST',insertionType]
    bedFile.close()

#Reading in coordinates from the following ERV insertion site bed file:
 /home/ampend/links/kidd-lab/ampend-projects/cfERVs/input/cfERV_NonReferenceInsertionSites_2017-06-29.bed
#There are a total of 59 cfERV insertions in this bed file

#Reading in coordinates from the following ERV insertion site bed file:
 /home/ampend/links/kidd-lab/ampend-projects/cfERVs/input/cfERV_ReferenceInsertionSites_2017-06-29.bed
#There are a total of 107 cfERV insertions in this bed file



In [5]:
def do_intersects(a, b, outfile):
    cmd = 'bedtools intersect -wo -a %s -b %s > %s' % (a, b, outfile)
    #print(cmd)
    runCMD(cmd)
    Dict = process_intersect(outfile)
    return Dict
########################################################################################
def process_intersect(outfile):
    Dict={}
    for line in open(outfile,'r'):
        line=line.rstrip().split()
        chrom,start,end,ID = line[0:4]
        hitID = line[7]
        if ID in Dict.keys():
            Dict[ID].append(hitID)
        else:
            Dict[ID] = []
            Dict[ID].append(hitID)
    return Dict
########################################################################################
def gene_intersects(a,b,intersectfile):
    cmd = 'bedtools intersect -wo -a %s -b %s > %s' % (a, b, outfile)
    runCMD(cmd)
    
    b2goDir = '/home/ampend/links/kidd-lab/ampend-projects/Angela/Re-RunningAnalysis_NewScripts/new_results/results/BLAST2GO/'
    # going through Ensembl bedtool intersect results file
    ensGDict = {} #ensGDict[windowID] = [gene1,gene2,gene3...]
    ensID = ''
    sigGenes = [] #To keep track of all gene IDs (ENSCAFG) that are in sig windows

    for line in open(intersectfile,'r'):
        line = line.rstrip().split('\t')
        winID = line[3]
        hit = line[7].split('_')
        Protein = hit[0]
        Gene = hit[1]
        b2goID = Gene + '|' + Protein
        ensGID = hit[2]
        
        if winID in ensGDict:
            ensGDict[winID].append(ensGID)
        else:
            ensGDict[winID] = []
            ensGDict[winID].append(ensGID) 
            sigGenes.append(b2goID)
    return ensGDict
########################################################################################
def do_window_intersects(a,b,windowSize,outfile):
    cmd = 'bedtools window -w %i -a %s -b %s > %s' % (windowSize, a, b, outfile)
    #print(cmd)
    runCMD(cmd)
    #Dict = process_intersect(outfile)
    #return Dict    

In [6]:
##### NON REFERENCE ######
ervfile = '/home/ampend/links/kidd-lab/ampend-projects/cfERVs/input/cfERV_NonReferenceInsertionSites_2017-06-29.bed'
ervDir = '/home/ampend/links/kidd-lab/ampend-projects/cfERVs/Insertion_Assessment/'


#Get XP-CLR intersects -- save to Dict
xpclr_file = '/home/ampend/links/kidd-lab/ampend-projects/Angela/XP-CLR/results/Campbell_No-MAF-Filter/50kbWindow_2kbGrid/Dogs_v_Wolves/SignificantWindows/Significant_3Windows_pct99_AverageXP-CLRScore.4.bed'
outfile = ervDir + 'Intersect_NonReferenceERVs_with_XP-CLRs_SimulationThreshold.txt'
xpclrDict = do_intersects(ervfile, xpclr_file, outfile)

#Get CDR intersects -- save to Dict
cdrfile = '/home/ampend/links/kidd-lab/ampend-projects/Angela/Re-RunningAnalysis_NewScripts/FINAL_results/MergedOverlapping_Sliding_TotalSIGFstCalls_AutoXPar_54callset.bed'
outfile = ervDir + 'Intersect_NonReferenceERVs_with_CDRs_SimulationThreshold.txt'
cdrDict = do_intersects(ervfile, cdrfile, outfile)

#Get ensembl gene intersects -- save to Dict
genebedfile = '/home/ampend/links/kidd-lab/ampend-projects/BLAST2GO/results/BLAST2GO_Ensembl81_GeneTables_WithEnscafIDsAndChrom.bed'
outfile = ervDir + 'Intersect_NonReferenceERVs_with_EnsemblGenes_SimulationThreshold.txt'
ensGDict = gene_intersects(ervfile, genebedfile, outfile)
#Get Axelsson loci intersects -- save to Dict
axelssonBedFile = '/home/ampend/links/kidd-lab/ampend-projects/Angela/Re-RunningAnalysis_NewScripts/new_results/results/Axelsson_CaganBlass/input/Axelsson_canfam3.1.bed'
outfile = ervDir + 'Intersect_NonReferenceERVs_with_AxelssonCDRs_SimulationThreshold.txt'
axDict = do_intersects(ervfile, axelssonBedFile, outfile)
#Get CaganBlass loci intersects -- save to Dict
caganBlassBedFile = '/home/ampend/links/kidd-lab/ampend-projects/Angela/Re-RunningAnalysis_NewScripts/new_results/results/Axelsson_CaganBlass/input/CaganBlass_canfam3.1.bed '
outfile = ervDir + 'Intersect_NonReferenceERVs_with_CaganBlassCDRs_SimulationThreshold.txt'
cbDict = do_intersects(xpclr_file, caganBlassBedFile, outfile)



In [7]:
print(bedfiles)

['/home/ampend/links/kidd-lab/ampend-projects/cfERVs/input/cfERV_NonReferenceInsertionSites_2017-06-29.bed', '/home/ampend/links/kidd-lab/ampend-projects/cfERVs/input/cfERV_ReferenceInsertionSites_2017-06-29.bed']


In [5]:
totalCDR_file = '/home/ampend/links/kidd-lab/ampend-projects/Angela/Re-RunningAnalysis_NewScripts/FINAL_results/MergedOverlapping_Sliding_TotalSIGFstCalls_AutoXPar_54callset.bed'

for File in bedfiles:
    erv_file = File
    ############################
    if 'Non' in erv_file:
        Type = 'NonReference'
    else:
        Type = 'Reference'
    ############################
    cmd = 'bedtools intersect -wo -a %s -b %s > %sresults/Intersect_cfERV_%s_with_MergedVCDRandCDRSet.txt' % (totalCDR_file, erv_file, ervDir,Type)
    print (cmd)
    runCMD(cmd)

    #How many intersect exactly?
    f = ervDir + 'results/Intersect_cfERV_%s_with_MergedVCDRandCDRSet.txt' % Type
    count = count_lines(f)
    print('-->%i %s cfERV insertions found to intersect merged CDR and VCDRs exactly\n' % (count,Type))

    cmd = 'bedtools window -w 50000 -a %s -b %s > %sresults/50kb_Intersect_cfERV_%s_with_MergedVCDRandCDRSet.txt' % (totalCDR_file, erv_file, ervDir,Type)
    print (cmd)
    runCMD(cmd)

    #How many intersect in windows of 50kb?
    f = ervDir + 'results/50kb_Intersect_cfERV_%s_with_MergedVCDRandCDRSet.txt' % Type
    count = count_lines(f)
    print ('-->%i %s cfERV insertions found to intersect merged CDR and VCDRs within 50kb\n' % (count, Type))



bedtools intersect -wo -a /home/ampend/links/kidd-lab/ampend-projects/Angela/Re-RunningAnalysis_NewScripts/FINAL_results/MergedOverlapping_Sliding_TotalSIGFstCalls_AutoXPar_54callset.bed -b /home/ampend/links/kidd-lab/ampend-projects/cfERVs/input/cfERV_NonReferenceInsertionSites_2017-06-29.bed > /home/ampend/links/kidd-lab/ampend-projects/cfERVs/results/Intersect_cfERV_NonReference_with_MergedVCDRandCDRSet.txt
-->3 NonReference cfERV insertions found to intersect merged CDR and VCDRs exactly

bedtools window -w 50000 -a /home/ampend/links/kidd-lab/ampend-projects/Angela/Re-RunningAnalysis_NewScripts/FINAL_results/MergedOverlapping_Sliding_TotalSIGFstCalls_AutoXPar_54callset.bed -b /home/ampend/links/kidd-lab/ampend-projects/cfERVs/input/cfERV_NonReferenceInsertionSites_2017-06-29.bed > /home/ampend/links/kidd-lab/ampend-projects/cfERVs/results/50kb_Intersect_cfERV_NonReference_with_MergedVCDRandCDRSet.txt
-->6 NonReference cfERV insertions found to intersect merged CDR and VCDRs within

In [9]:
olfactory_file = '/home/ampend/links/kidd-lab-scratch/feichens-projects/dogs/olfactory/OR_all_sort_noU.bed'
erv_file = ervDir + 'input/' + 'cfERV_InsertionSites_2016-12-06.txt'
for File in bedfiles:
    erv_file = File
    ############################
    if 'Non' in erv_file:
        Type = 'NonReference'
    else:
        Type = 'Reference'
    ############################
    
    cmd = 'bedtools intersect -wo -a %s -b %s > %sresults/Intersect_cfERV_with_OlfactoryReceptors.txt' % (olfactory_file, erv_file, ervDir)
    runCMD(cmd)

    #How many intersect exactly?
    f = ervDir + 'results/Intersect_cfERV_with_OlfactoryReceptors.txt'
    count = count_lines(f)
    print ('-->%i cfERV %s insertions found to intersect olfactory receptors exactly' % (count,Type))

    #Within 5kb?
    cmd = 'bedtools window -w 5000 -a %s -b %s > %sresults/5kb_Intersect_cfERV_with_OlfactoryReceptors.txt' % (olfactory_file, erv_file, ervDir)
    runCMD(cmd)

    #How many intersect Within 50kb??
    f = ervDir + 'results/5kb_Intersect_cfERV_with_OlfactoryReceptors.txt'
    count = count_lines(f)
    print ('-->%i cfERV %s insertions found to intersect olfactory receptors within 5kb' % (count,Type))


    #Within 50kb?
    cmd = 'bedtools window -w 50000 -a %s -b %s > %sresults/50kb_Intersect_cfERV_with_OlfactoryReceptors.txt' % (olfactory_file, erv_file, ervDir)
    runCMD(cmd)

    #How many intersect Within 50kb??
    f = ervDir + 'results/50kb_Intersect_cfERV_with_OlfactoryReceptors.txt'
    count = count_lines(f)
    print ('-->%i cfERV %s insertions found to intersect olfactory receptors within 50kb' % (count,Type))



-->0 cfERV NonReference insertions found to intersect olfactory receptors exactly
-->1 cfERV NonReference insertions found to intersect olfactory receptors within 5kb
-->9 cfERV NonReference insertions found to intersect olfactory receptors within 50kb
-->0 cfERV Reference insertions found to intersect olfactory receptors exactly
-->0 cfERV Reference insertions found to intersect olfactory receptors within 5kb
-->0 cfERV Reference insertions found to intersect olfactory receptors within 50kb


In [20]:
def determine_file_type(erv_file):
    if 'Non' in erv_file:
        Type = 'NonReference'
    else:
        Type = 'Reference'
    print('\n#######\n' + Type + '\n#######\n')
    return Type
##############################################################################################################
def build_ensGDict(erv_file):
    inFile = open(erv_file,'r')
    for line in inFile:
        line=line.rstrip().split('\t')
        winID=line[3]
        ensGDict[winID] = [[],[],[],[],[]]
    return ensGDict
##############################################################################################################
def intersect_with_FST_CDRs(ervDir,erv_file,windowLength):
    #bedfile with the final FST CDR coordinates
    fstfile = '/home/ampend/links/kidd-lab/ampend-projects/Angela/Re-RunningAnalysis_NewScripts/FINAL_results/MergedOverlapping_Sliding_TotalSIGFstCalls_AutoXPar_54callset.bed'

    #Intersecting the vst BED file with the FST results
    outfile = ervDir + 'results/' + windowLength + 'kbWindowIntersect_%s_cfERVs_with_FstCDRs.txt' % Type
    print ('\n#Intersecting the FST and VST windows with %skb buffering windows...' % windowLength)
    cmd = 'bedtools window -w %s -a %s -b %s > %s' % (windowLength,erv_file, fstfile, outfile)
    runCMD(cmd)

    #checking window counts
    f = outfile
    count = count_lines(f)
    print ('-->%i cfERV insertions found to intersect FST regions with window size = %skb' % (count,windowLength))
##############################################################################################################
def intersect_with_XPCLR(ervDir,erv_file,windowLength):
    #bedfile with the final FST CDR coordinates
    fstfile = '/home/ampend/links/kidd-lab/ampend-projects/Angela/XP-CLR/results/Campbell_No-MAF-Filter/50kbWindow_2kbGrid/Dogs_v_Wolves/SignificantWindows/Significant_3Windows_pct99_AverageXP-CLRScore.4.bed'

    #Intersecting the vst BED file with the FST results
    outfile = ervDir + 'results/' + windowLength + 'kbWindowIntersect_%s_cfERVs_with_XPCLRs.txt' % Type
    print ('\n#Intersecting the FST and VST windows with %skb buffering windows...' % windowLength)
    cmd = 'bedtools window -w %s -a %s -b %s > %s' % (windowLength,erv_file, fstfile, outfile)
    print(cmd)
    runCMD(cmd)

    #checking window counts
    f = outfile
    count = count_lines(f)
    print ('-->%i cfERV insertions found to intersect XP-CLR regions with window size = %skb' % (count,windowLength))


##############################################################################################################
def intersect_with_Ensembl_Genes(bedfile,windowLength):
    #Determining variables
    inBedFile = bedfile
    genebedfile = '/home/ampend/links/kidd-lab/ampend-projects/BLAST2GO/results/BLAST2GO_Ensembl81_GeneTables_WithEnscafIDsAndChrom.bed'
    print( '#Reading in the Ensembl 81 genes from the following file:\n', genebedfile)
    
    #Intersecting with Ensembl bed files
    outfile = ervDir + 'results/' + windowLength + 'kbWindowIntersect_%s_cfERVs_with_Ensembl81Genes.txt' % Type
    print ('\n#Intersecting the Ensembl genes with %skb buffering windows...' % windowLength)
    cmd = 'bedtools window -w %s -a %s -b %s > %s' % (windowLength,inBedFile, genebedfile, outfile)
    runCMD(cmd)

    #checking window counts
    f = outfile
    count = count_lines(f)
    print ('-->%i cfERV insertions found to intersect Ensembl 81 genes with window size = %skb' % (count,windowLength))
##############################################################################################################
def intersect_with_Ensembl_FullGenes(bedfile,windowLength):
    #Determining variables
    inBedFile = bedfile
    genebedfile = '/home/ampend/links/kidd-lab/ampend-projects/BLAST2GO/ensembl_input/ensembl-81-genes.bed'
    print( '#Reading in the full seq Ensembl 81 genes from the following file:\n', genebedfile)
    
    #Intersecting with Ensembl bed files
    outfile = ervDir + 'results/' + windowLength + 'kbWindowIntersect_%s_cfERVs_with_FullLengthEnsembl81Genes.txt' % Type
    print ('\n#Intersecting the Ensembl genes with %skb buffering windows...' % windowLength)
    cmd = 'bedtools window -w %s -a %s -b %s > %s' % (windowLength,inBedFile, genebedfile, outfile)
    runCMD(cmd)

    #checking window counts
    f = outfile
    count = count_lines(f)
    print ('-->%i cfERV insertions found to intersect Full length Ensembl 81 genes with window size = %skb' % (count,windowLength))
##############################################################################################################
def intersect_with_other_refseq(bedfile,windowLength):
    #Determining variables
    inBedFile = bedfile
    genebedfile = '/home/ampend/links/kidd-lab/ampend-projects/cfERVs/GeneIntersects/other_refseq.bed'
    
    #Intersecting with Ensembl bed files
    outfile = ervDir + 'results/' + windowLength + 'kbWindowIntersect_%s_cfERVs_with_OtherRefSeqGenes.txt' % Type
    print ('\n#Intersecting the "Other" RefSeq with %skb buffering windows...' % windowLength)
    cmd = 'bedtools window -w %s -a %s -b %s > %s' % (windowLength,inBedFile, genebedfile, outfile)
    runCMD(cmd)

    #checking window counts
    f = outfile
    count = count_lines(f)
    print ('-->%i cfERV insertions found to intersect Full length "Other" RefSeq (from UCSC) genes with window size = %skb' % (count,windowLength))
##############################################################################################################
def intersect_with_Dog_EST(bedfile,windowLength):
    #Determining variables
    inBedFile = bedfile
    genebedfile = '/home/ampend/links/kidd-lab/ampend-projects/cfERVs/GeneIntersects/dog_allESTs_CF3.bed'
    
    #Intersecting with Ensembl bed files
    outfile = ervDir + 'results/' + windowLength + 'kbWindowIntersect_%s_cfERVs_with_DogESTs.txt' % Type
    print ('\n#Intersecting the Dog ESTs with %skb buffering windows...' % windowLength)
    cmd = 'bedtools window -w %s -a %s -b %s > %s' % (windowLength,inBedFile, genebedfile, outfile)
    runCMD(cmd)

    #checking window counts
    f = outfile
    count = count_lines(f)
    print ('-->%i cfERV insertions found to intersect Dog ESTs (from UCSC) genes with window size = %skb' % (count,windowLength))


In [27]:
def parse_FST_CDR_intersect_file():
    #Reading the intersect file results to FST CDRs
    outfile = ervDir + 'results/' + 'kbWindowIntersect_cfERV_%s_with_MergedVCDRandCDRSet.txt' % Type
    intersectFile = open(outfile,'r') #outfile from the intersect step above
    # going through Ensembl bedtool intersect results file
    fstDict = {} #ensGDict[windowID] = [gene1,gene2,gene3...]
    fstID = ''
    fstWindows = [] #To keep track of all gene IDs (ENSCAFG) that are in sig windows

    for line in intersectFile:
        line = line.rstrip() #removing extraneous whitespace characters
        line = line.split() #delimiting "columns" in the file based on tabs		
        vstID = line[3]
        fstID = line[8]

        if vstID in fstDict:
            fstDict[vstID].append(fstID)
        else:
            fstDict[vstID] = []
            fstDict[vstID].append(fstID) 
    intersectFile.close()
    return fstDict
##############################################################################################################
def parse_FST_XPCLR_intersect_file():
    #Reading the intersect file results to FST CDRs
    outfile = ervDir + 'results/' + 'kbWindowIntersect_cfERV_%s_with_XPCLRs.txt' % Type
    print(outfile)
    intersectFile = open(outfile,'r') #outfile from the intersect step above
    # going through Ensembl bedtool intersect results file
    fstDict = {} #ensGDict[windowID] = [gene1,gene2,gene3...]
    fstID = ''
    fstWindows = [] #To keep track of all gene IDs (ENSCAFG) that are in sig windows

    for line in intersectFile:
        line = line.rstrip() #removing extraneous whitespace characters
        line = line.split() #delimiting "columns" in the file based on tabs		
        vstID = line[3]
        fstID = line[7]

        if vstID in fstDict:
            fstDict[vstID].append(fstID)
        else:
            fstDict[vstID] = []
            fstDict[vstID].append(fstID) 
    intersectFile.close()
    return fstDict
##############################################################################################################
def parse_gene_intersect_file(windowLength,index):
    intfile = ervDir + 'results/' + windowLength + 'kbWindowIntersect_%s_cfERVs_with_Ensembl81Genes.txt' % Type
    intersectFile = open(intfile,'r') #outfile from the intersect step with ensembl 81 genes

    b2goDir = '/home/ampend/links/kidd-lab/ampend-projects/Angela/Re-RunningAnalysis_NewScripts/new_results/results/BLAST2GO/'
    b2goList = open(ervDir + 'results/' + windowLength + '_cfERV_' + Type + '_GeneList.txt', 'w')

    ensID = ''
    sigGenes = [] #To keep track of all gene IDs (ENSCAFG) that are in sig windows

    #Reading the intersect file results
    for line in intersectFile:
        line = line.rstrip() #removing extraneous whitespace characters
        line = line.split() #delimiting "columns" in the file based on tabs		
        winID = line[3]
        hit = line[7].split('_')
        Protein = hit[0]
        Gene = hit[1]
        b2goID = Gene + '|' + Protein
        ensGID = hit[2]
        
        if winID in ensGDict:
            ensGDict[winID][index].append(ensGID)
        else:
            ensGDict[winID][index].append(ensGID) 
            sigGenes.append(b2goID)
    
    intersectFile.close()
    for i in sigGenes:
        gene = i
        b2goList.write('%s\n' % gene)
    
    return ensGDict
##############################################################################################################
def parse_fullgene_intersect_file(windowLength,index):
    full_ensGDict={}
    full_ensGDict=build_ensGDict(erv_file)

    intfile = ervDir + 'results/' + windowLength + 'kbWindowIntersect_%s_cfERVs_with_FullLengthEnsembl81Genes.txt' % Type
    intersectFile = open(intfile,'r') #outfile from the intersect step with ensembl 81 genes

    b2goDir = '/home/ampend/links/kidd-lab/ampend-projects/Angela/Re-RunningAnalysis_NewScripts/new_results/results/BLAST2GO/'
    b2goList = open(ervDir + windowLength + '_cfERV_' + Type + '_FullLengthGeneList.txt', 'w')

    ensID = ''
    sigGenes = [] #To keep track of all gene IDs (ENSCAFG) that are in sig windows

    #Reading the intersect file results
    for line in intersectFile:
        line = line.rstrip() #removing extraneous whitespace characters
        line = line.split() #delimiting "columns" in the file based on tabs		
        winID = line[3]
        ensGID = line[7]

        if winID in full_ensGDict:
            full_ensGDict[winID][index].append(ensGID)
        else:
            full_ensGDict[winID][index].append(ensGID) 
            sigGenes.append(b2goID)    
    return full_ensGDict
##############################################################################################################
def parse_intersect_file(windowLength,intfile):
    Dict = {}
    intersectFile = open(intfile,'r') #outfile from the intersect step with ensembl 81 genes

    sigHits = [] #To keep track of all hits that are in sig windows
    #Reading the intersect file results
    for line in intersectFile:
        line = line.rstrip() #removing extraneous whitespace characters
        line = line.split() #delimiting "columns" in the file based on tabs
        winID, hitID = line[3], line[7]

        if winID in Dict:
            Dict[winID].append(hitID)
        else:
            Dict[winID] = []
            Dict[winID].append(hitID) 
            sigHits.append(hitID)    
    intersectFile.close()
    return Dict
##############################################################################################################

In [23]:
################################################################################################################
def making_summary_text_files(Type,fstDict,ensGDict,erv_file,otherDict,estDict):
    inFile = open(erv_file, 'r')
    ############################################################################
    # SUMMARIZING ERV INSERTIONS WITH ENSEMBL GENES AND FST REGIONS
    ##########################################################################
    Outfile = ervDir + 'results/' + Type + '_cfERV_WithFSTIntersects_GeneTable.txt' #Write out the VST Summary data table here
    outFile = open(Outfile, 'w')
    outFile.write('Chrom\tStart\tEnd\tWindowID\tERV_Type\tIntersecting_FST_CDR\t0kb_IntersectingExons\t2.5kb_IntersectingExons\t5kb_IntersectingExons\t10kb_IntersectingExons\t50kb_IntersectingExons\t0kb_IntersectingIntrons\t0kb_Intersecting"Other"RefSeqGenes\t0kb_IntersectingDogESTs\n')

    allDict = {} # defining the dictionary that will have all the results of bedtool intersect compiled by windows

    for line in inFile:
        line = line.rstrip()
        line = line.split('\t')
        chrom, start_pos, end_pos, WinID = line[0:4]
        allDict[WinID] = [chrom,start_pos,end_pos,WinID,Type,False,False,False,False,False,False,False,False,False] #setting up empty dictionary
        #Open up dictionary for ERVs
        #0=chr, 1=start_pos, 2=end_pos, 3=ERV ID 4=ERV Type (reference/nonreference)
        #5=Intersects with an FST CDR (default = False), 
        #6=Intersects with Ensembl Gene IDs (default = False) at 0kb
        #7=Intersects with Ensembl Gene IDs (default = False) at 2.5kb
        #8=Intersects with Ensembl Gene IDs (default = False) at 5kb
        #9=Intersects with Ensembl Gene IDs (default = False) at 10kb
        #10=Intersects with Ensembl Gene IDs (default = False) at 50kb
        #11=Intersects with FULL LENGTH Ensembl Gene IDs (default = False) at 0kb
        #12 = Intersects "Other" Refseq Genes (from UCSC) at 0kb
        #13 = Intersects Dog EST (from UCSC) at 0kb
        
        #Did this ERV window intersect with a FST CDR?
        if WinID in fstDict: #If this window is in the ensembl list you generated, then...:
            y = ", ".join(map(str, fstDict[WinID])) #This takes the list you generated from the Ensembl file and joins each entry in the list with a comma
            allDict[WinID][5] = y #Now it adds 'y' you generated in the line above to your dictionary

        #Did this ERV intersect with a gene?
        if WinID in ensGDict: #If this window is in the ensembl list you generated, then...:
            for num in range(0,index+1):
                if len(ensGDict[WinID][num]) > 0:
                    y = ", ".join(map(str, ensGDict[WinID][num])) #This takes the list you generated from the Ensembl file and joins each entry in the list with a comma
                    allDict[WinID][6+num] = y #Now it adds 'y' you generated in the line above to your dictionary

        #Did this ERV intersect with a gene?
        if WinID in full_ensGDict: #If this window is in the ensembl list you generated, then...:
            #if len(full_ensGDict[WinID][0]) > 0:
            #    print(full_ensGDict[WinID][0],'#####',ensGDict[WinID][0])
            #else:
            #    continue
            geneID=full_ensGDict[WinID][0]
            exon_geneID = ensGDict[WinID][0]
            if geneID not in exon_geneID:
                allDict[WinID][11] = geneID

        #Did this ERV window intersect with an "OTHER" RefSeq gene?
        if WinID in otherDict: #If this window is in the ensembl list you generated, then...:
            y = ", ".join(map(str, otherDict[WinID])) #This takes the list you generated from the Ensembl file and joins each entry in the list with a comma
            allDict[WinID][12] = y #Now it adds 'y' you generated in the line above to your dictionary

        #Did this ERV window intersect with an "OTHER" RefSeq gene?
        if WinID in estDict: #If this window is in the ensembl list you generated, then...:
            y = ", ".join(map(str, estDict[WinID])) #This takes the list you generated from the Ensembl file and joins each entry in the list with a comma
            allDict[WinID][13] = y #Now it adds 'y' you generated in the line above to your dictionary
        
    #Writing out results
    for keys in sorted(allDict.keys()): #Now loops through each 
        outFile.write("\t".join(map(str,allDict[keys])))
        outFile.write("\n")
    outFile.close()
    inFile.close()


In [26]:
ervDir = '/home/ampend/links/kidd-lab/ampend-projects/cfERVs/'

for erv_file in bedfiles:
    intersectWindows = ['0','2500','5000','10000','50000']
    Type, index = determine_file_type(erv_file), -1
    ensGDict = {} #ensGDict[windowID] = [gene1,gene2,gene3...]
    ensGDict=build_ensGDict(erv_file)
    
    fstDict=build_ensGDict(erv_file)
    
    full_ensGDict={}
    full_ensGDict=build_ensGDict(erv_file)
    
    for windowLength in intersectWindows:
        index+=1
        print('\n####Window length: ', windowLength)
        #1. Intersecting with FST CDRs
        
        intersect_with_FST_CDRs(ervDir,erv_file,windowLength)
        intersect_with_XPCLR(ervDir,erv_file,windowLength)
        
        #2. Intersecting with Ensembl Genes
        #print ('\n##2. Now intersecting VST regions with Ensembl 81 genes')
        intersect_with_Ensembl_Genes(erv_file,windowLength)    
        intersect_with_Ensembl_FullGenes(erv_file,windowLength)
               
        #3. Parse bedtools intersect file against Ensembl genes
        ensGDict = parse_gene_intersect_file(windowLength,index)
        full_ensGDict = parse_fullgene_intersect_file(windowLength,index)

        #4. Parse bedtools intersect file against FST CDRs
        fstDict = parse_FST_CDR_intersect_file()
        xpclrDict = parse_FST_XPCLR_intersect_file()
    
    
    windowLength = '0'
    #5. Intersecting and Parsing results with "Other" RefSeq UCSC Track
    intersect_with_other_refseq(erv_file,windowLength)
    intfile = ervDir + 'results/' + windowLength + 'kbWindowIntersect_%s_cfERVs_with_OtherRefSeqGenes.txt' % Type
    otherDict = parse_intersect_file(windowLength,intfile)

    #6. Intersecting and Parsing results with TransMap EST UCSC Track
    intersect_with_Dog_EST(erv_file,windowLength)
    intfile = ervDir + 'results/' + windowLength + 'kbWindowIntersect_%s_cfERVs_with_DogESTs.txt' % Type
    estDict = parse_intersect_file(windowLength,intfile)

    ###########################################################
    #7. Summarizing VST and Ensembl hits to output text files
    ###########################################################
    print ('\n##7. Now generating summary text files for %s results...' % (Type))
    allDict = making_summary_text_files(Type,fstDict,ensGDict,erv_file,otherDict,estDict)
    


#######
NonReference
#######


####Window length:  0

#Intersecting the FST and VST windows with 0kb buffering windows...
-->3 cfERV insertions found to intersect FST regions with window size = 0kb

#Intersecting the FST and VST windows with 0kb buffering windows...
bedtools window -w 0 -a /home/ampend/links/kidd-lab/ampend-projects/cfERVs/input/cfERV_NonReferenceInsertionSites_2017-06-29.bed -b /home/ampend/links/kidd-lab/ampend-projects/Angela/XP-CLR/results/Campbell_No-MAF-Filter/50kbWindow_2kbGrid/Dogs_v_Wolves/SignificantWindows/Significant_3Windows_pct99_AverageXP-CLRScore.4.bed > /home/ampend/links/kidd-lab/ampend-projects/cfERVs/results/0kbWindowIntersect_NonReference_cfERVs_with_XPCLRs.txt
-->0 cfERV insertions found to intersect XP-CLR regions with window size = 0kb
#Reading in the Ensembl 81 genes from the following file:
 /home/ampend/links/kidd-lab/ampend-projects/BLAST2GO/results/BLAST2GO_Ensembl81_GeneTables_WithEnscafIDsAndChrom.bed

#Intersecting the Ensembl genes wit

FileNotFoundError: [Errno 2] No such file or directory: '/home/ampend/links/kidd-lab/ampend-projects/cfERVs/results/kbWindowIntersect_cfERV_NonReference_with_MergedVCDRandCDRSet.txt'

In [10]:
################################################################################################################
def gene_fst_intersect_summary(allDict):
    geneCount = 0
    fstCount = 0
    for keys in sorted(allDict.keys()):
        if allDict[keys][6] is not False:
            fstCount += 1
        if allDict[keys][7] is not False:
            geneCount += 1

    print ('%i windows intersect with FST CDRs' % (fstCount))
    print ('%i windows intersect with Ensembl genes' % (geneCount))
############################################################################################################
def make_gene_enrichment_list(Type):
    enrFile = open(ervDir + 'results/cfERV_' + Type + '_GeneEnrichmentList.txt')
    for winID in allDict.keys():
        print 